# Machine Learning Applications for Health (COMP90089_2024_SM2)
# Tutorial: DATA Wrangling

### TOPICS:

*   Missing Values
*   Outliers
*   Standardisation


### DATA SET:

* Extracted from MIMIC-II-demo: ICU stay details with comorbity and admission information 


### LIBRARIES:

* pandas
* numpy
* seaborn
* matplotlib


---

### Context: 

The MIMIC II demo clinical database, consists of data integrated from diferent information systems in the hospital and contains diverse information such as: patient demographics, medications, results of lab tests and more. It is a much earlier and smaller version of the data that is now contained in MIMIC-IV, which will be the primary focus of this subject. I've extracted some data from this database and saved it as cSV files which will be read in with Pandas.



## Import Libraries and uncomment the 'pip install' rows if any prior installation is necessary.

In [ ]:
#!pip install seaborn
#!pip install pandas
#!pip install numpy
#!pip install matplotlib

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#Display maximum columns on the screen
pd.set_option('display.max_columns', None)

#### Let's look at some comorbidity data

For reference, this data was originally obtained using the following query:

```sql
SELECT *
FROM mimic2.comorbidity_scores
```

In [ ]:

comorbidity = pd.read_csv("https://raw.githubusercontent.com/melbourne-cdth/comp90089_tutorial2/main/comorbidity.csv")
#Print the output and the for the available column names

print(comorbidity.columns)
comorbidity.head(20)

### Richer data set

Now that we have an idea of how the table looks like, let's gather more information from parts of MIMIC to make the data richer in details.

* **First table** to consider: icustay_detail (rename it icu to make it shorter)

We will then join information from **2 other Tables** based on 1 mutual criteria.
* Second Table: comorbidity_scores
* Third Table: admissions

**Criteria:** The Hospital admission id must be the same (column hadm_id). This is an integer number identifying an ICU stay.

**Note** that we are now specifying the columns we want to retrieve from each table, except for the comorbidity_score that we want to retrieve all columns.

The query used to get this data was as follows:

```sql
SELECT comorb.*, adm.admit_dt, adm.disch_dt, icu.gender, icu.dod, icu.expire_flg, icu.hospital_los, icu.icustay_los, icu.sapsi_first, icu.sapsi_max, icu.sofa_first, icu.sofa_max
FROM mimic2.icustay_detail as icu
INNER JOIN mimic2.comorbidity_scores as comorb 
ON comorb.hadm_id = icu.hadm_id
INNER JOIN mimic2.admissions as adm
ON adm.hadm_id = icu.hadm_id
ORDER BY subject_id
```

In [ ]:

result_merged = pd.read_csv("https://raw.githubusercontent.com/melbourne-cdth/comp90089_tutorial2/main/comorbidity_v2.csv")
#Print the output and the for the available column names
print(result_merged.columns)
result_merged.head(10)

## **Type of data: Numerical or Categorical**

In [ ]:
## Check the type of data(Numerical or Categorical)
## Dtype in this set: float64, int64, object

result_merged.info()

categ_columns = [col for col in result_merged.columns if result_merged[col].dtype=='object']
numerical_columns = [col for col in result_merged.columns if (result_merged[col].dtype=='float64' or result_merged[col].dtype=='int64')]

print("\n Categorical features in this data set: ",categ_columns,"\n")
print("\n Numerical features in this data set: ",numerical_columns,"\n")

In [ ]:
## View some statistical measurements of this data (only for numerical columns):

result_merged.describe()

## Checking for Missing values


In [ ]:
##Missing values
result_merged.isnull().sum()

## Checking for Outliers

* Outliers are observations that deviate significantly from other values in a dataset.
* There are many ways to classify a numerical value as an outlier, here we will see one based on the Interquartile range.

In [ ]:
# Function to Detection Outlier on one-dimentional datasets.
# IQR based filtering  (Interquartile Range)

def find_outliers(df, col_name):   
    percentile25 = result_merged[col_name].quantile(0.25)
    percentile75 = result_merged[col_name].quantile(0.75)
    
    iqr = percentile75-percentile25
    upper_limit = percentile75 + 1.5 * iqr
    lower_limit = percentile25 - 1.5 * iqr
    
    anomalies_upper = result_merged[col_name][result_merged[col_name] > upper_limit]
    anomalies_lower = result_merged[col_name][result_merged[col_name] < lower_limit]
    return_file_up = pd.DataFrame(anomalies_upper)
    return_file_lw = pd.DataFrame(anomalies_lower)

    if anomalies_upper.size != 0 and anomalies_lower.size != 0:
      anomalies_frame = pd.concat([return_file_up,return_file_lw])

    elif anomalies_lower.size != 0:
      anomalies_frame = return_file_lw

    elif anomalies_upper.size != 0:
      anomalies_frame = return_file_up
      
    return anomalies_frame


In [ ]:
## Length of stay in ICU (icustay_los) Column Outliers based on IQR Filtering:
find_outliers(result_merged,'icustay_los')

## Replacing values

* One of the ways to deal with missing values or outliers is to remove the values with too many missing variables using a threshold.
* Or to replace it with some statistical measurement (mean, median, quantile...)

But in all cases you should evaluate if the strategy will make sense to the data or not. Outliers can also be rare information underlying the data and no action will be require on them.

In [ ]:
## Numerical Columns with missing values: sapsi_first,sapsi_max,sofa_first,sofa_max

## Let's see how to replace missing values for sapsi_first by the Quartile 50.

quart50_sapsi_first = result_merged['sapsi_first'].quantile(0.5)
print("Value of Quantile 50% for sapsi_first column: ",quart50_sapsi_first,"\n")

print("Rows with NaN values in sapsi_first column: \n",result_merged['sapsi_first'][result_merged['sapsi_first'].isna()],"\n")

# Replace the NaN values with the Quartile 50. Check them again.
result_merged['sapsi_first'].fillna(quart50_sapsi_first, inplace=True)
print("Rows with NaN values in sapsi_first column: ",result_merged['sapsi_first'][result_merged['sapsi_first'].isna()],"\n")

In [ ]:
result_merged.head()

## Transforming Categorical into Numbers:

We will see in the next Tutorials that, when dealing with Machine Learning, we ideally transform categorical data into Numbers.

* Categorical columns in this data set:  ['category', 'admit_dt', 'disch_dt', 'gender', 'dod', 'expire_flg'] 
* The method **get_dummies** from pandas library can convert the columns with object or category dtype into numerical.

In [ ]:
# Transform the Column Gender using get_dummies. It will create 2 new columns with binary values for Male or Female.

gender_categorical = pd.get_dummies(result_merged['gender'])
print(gender_categorical)

## Compute pairwise correlation of columns

In [ ]:
#Exclude unecessary columns first
result_merged_clean = result_merged.drop(['subject_id','category', 'gender', 'expire_flg','hadm_id','admit_dt','disch_dt','dod'], axis = 1)

#Create the correlation variable. The .coor() method has some default parameters.
correlation = result_merged_clean.corr() ##default method='pearson'

#Print the correlation output values
print(correlation)


In [ ]:
# Plot the output figure
plt.figure(figsize= (20,20))
sns.heatmap(correlation, annot=True)
plt.show()